In [259]:
%load_ext autoreload
%autoreload 2
import sqlite3
import pandas as pd
import numpy as np
from datetime import datetime
from srs import *

In [349]:
!rm srs.db
!python3 create_srs.py
!python3 fill_srs.py

In [350]:
srs.close()
srs = SRS()

In [334]:
datetime.fromtimestamp(1582085951)

datetime.datetime(2020, 2, 18, 23, 19, 11)

In [372]:
datetime.fromtimestamp(1582172665)

datetime.datetime(2020, 2, 19, 23, 24, 25)

In [377]:
srs.show_db()

CONCEPTS


,cid,created_date,interval,last_review,due_date
0,1,1582086139,7,1.582086e+09,1582691229
1,2,1582086139,0,1.582086e+09,1582086270


QUESTIONS


,qid,created_date,cid,front,back,num_reviews,last_review
0,1,1582086139,1,the front,the back,1,1.582086e+09
1,2,1582086139,1,the dearre,the baawfeaweck,3,1.582086e+09
2,3,1582086139,1,the,the b,0,0.000000e+00
3,4,1582086139,2,capital of france?,paris,1,1.582086e+09
4,5,1582086139,2,paris is the capital of what?,france,0,0.000000e+00
5,6,1582086139,2,france's capital is?,paris,1,1.582086e+09


REVIEWS


,rid,qid,cid,start_time,end_time,elapsed_time,passed
0,1,1,1,1.582086e+09,1.582086e+09,0.601173,0
1,2,6,2,1.582086e+09,1.582086e+09,0.213420,1
2,3,2,1,1.582086e+09,1.582086e+09,9.440872,1
3,4,4,2,1.582086e+09,1.582086e+09,1.125155,0
4,5,2,1,1.582086e+09,1.582086e+09,9.440872,1
5,6,2,1,1.582086e+09,1.582086e+09,9.440872,1


In [362]:
session = srs.start_session('2020-02-20')

In [366]:
session.start()

capital of france?


In [367]:
session.end()

paris

Did you get it?


In [368]:
session.no()

Done!


In [373]:
q = session.questions[0]

In [ ]:
q.__dic

In [376]:
srs._add_review(**q.dump())

In [234]:
q = "DROP TRIGGER IF EXISTS update_interval;"
srs.c.execute(q)
srs.conn.commit()

In [235]:
q = """
create trigger update_interval
    after insert on reviews
begin
    update concepts 
    set interval = case
        when new.passed = 0 then 0
        else 2*interval + 1
    END
    where cid = new.cid;
end;
"""
srs.c.execute(q)
srs.conn.commit()

In [85]:
session.questions[1].__dict__

{'qid': 5,
 'cid': 2,
 'front': 'paris is the capital of what?',
 'back': 'france',
 'num_reviews': 0,
 'start_time': 1582080958.291575,
 'end_time': 1582080958.942982,
 'passed': True}

In [39]:
qs = srs.get_due_questions('2020-02-20')

In [27]:
srs.add_question(2, 'Paris is in which country?', 'France')

In [229]:
conn.commit()

- Start review session
- Give me the next card
- Start review
- End review
- Mark as pass/fail

- Add card to database
- Add question 1 to card
- Add question 2 to card
- ...
- Review card
- Select fail/pass
- The review is logged with:
    - The timestamp of the start and end of review
    - The time to complete the review
    - Whether the review was a pass or fail
- The question is updated:
    - The last_review timestamp is updated
    - The number of reviews are updated
- The card is updated
    - The last_review timestamp is updated
    - The number of reviews are updated
    - The interval is updated


In [230]:
show_db()

CONCEPTS


,cid,created_date,interval,last_review,due_date
0,1,2020-02-19 01:50:43,0,NaN,2020-02-19 01:50:43
1,2,2020-02-19 01:50:43,0,1.582077e+09,2020-02-19 01:50:43


QUESTIONS


,qid,created_date,cid,front,back,num_reviews,last_review
0,1,2020-02-19 01:50:43,1,the front,the back,0,NaN
1,2,2020-02-19 01:50:43,1,the dearre,the baawfeaweck,1,1.582077e+09
2,3,2020-02-19 01:50:43,1,the,the b,2,NaN
3,4,2020-02-19 01:50:43,2,capital of france?,paris,0,NaN
4,5,2020-02-19 01:50:43,2,paris is the capital of what?,france,0,NaN
5,6,2020-02-19 01:50:43,2,france's capital is?,paris,2,NaN


REVIEWS


,rid,qid,cid,start_time,end_time,elapsed_time
0,1,2,2,1.582077e+09,1.582077e+09,10.0


In [208]:
q = """
select *
from questions 
group by cid
having min(num_reviews)
order by num_reviews
"""
pd.read_sql(q, conn)

,qid,created_date,cid,front,back,num_reviews,last_review


In [255]:
q = """
select *
from (
    select *
    from questions 
)
group by cid
having min(random())
"""
pd.read_sql(q, conn)

,qid,created_date,cid,front,back,num_reviews,last_review
0,1,2020-02-19 01:50:43,1,the front,the back,0,None
1,6,2020-02-19 01:50:43,2,france's capital is?,paris,2,None


### select the cards to review today

In [ ]:
date = '2020-02-20'
q = f"""
select q.qid, q.cid, q.front, q.back, q.num_reviews
-- 1) select questions whose concept is due
from (
    select q.*
    from questions q, concepts c
    where q.cid = c.cid and c.due_date <= '{date}'
) q
-- 2) select questions with the fewest reviews
join (
    select cid, min(num_reviews) num_rev
    from questions
    group by cid
) min
on 
    q.num_reviews = min.num_rev 
    and q.cid = min.cid
-- 3) randomly select 1 question from each concept
group by q.cid
having min(random())
"""
# pd.read_sql(q, conn)
c.execute(q)
records = c.fetchall()
cards = [Card(*r) for r in records]

### Review the cards

In [282]:
from datetime import datetime

In [285]:
datetime.now().timestamp()

1582077853.654437

In [33]:
class Question:
    def __init__(self, qid, cid, front, back, num_reviews):
        self.qid = qid
        self.cid = cid
        self.front = front
        self.back = back
        self.num_reviews = num_reviews
        
    def start(self):
        self.start_time = datetime.now().timestamp()
        print(self.front)
        
    def end(self):
        self.end_time = datetime.now().timestamp()
        print(self.back)
        
    def yep(self):
        self.passed = True
        
    def nope(self):
        self.passed = False

In [292]:
questions = [Question(*r) for r in records]

In [302]:
qiter = iter(questions)

In [311]:
q = next(qiter)

StopIteration: 

In [93]:
!rm srs.db

In [308]:
q.start()

paris is the capital of what?


In [309]:
q.end()

france


In [310]:
q.yep()

In [301]:
q.__dict__

{'qid': 1,
 'cid': 1,
 'front': 'the front',
 'back': 'the back',
 'num_reviews': 0,
 'start_time': 1582078068.507053,
 'end_time': 1582078069.39564,
 'passed': True}

Add review to log: start, end, elapsed

Trigger to increment num_reviews in QUESTIONS

Trigger to increment last_reviewed in QUESTIONS and CARDS

Update card in the database (should this be triggered by the log addition?)

In [295]:
q = f"""
update questions 
set  
    num_reviews={card.num_reviews}
where qid={card.qid}
"""
c.execute(q)

In [296]:
show_db()

CARDS


,cid,created_date,interval,due_date
0,1,2020-02-14 03:33:57,0,2020-02-14 03:33:57


QUESTIONS


,qid,created_date,cid,front,back,num_reviews
0,1,2020-02-14 03:33:57,1,the front,the back,2
1,2,2020-02-14 03:33:57,1,the dearre,the baawfeaweck,3
2,3,2020-02-14 03:33:57,1,the,the b,2


# run gui

In [121]:
from tkinter import *



def create_main_window():
    root = Tk()
    root.title('SRS')
    root.geometry('400x400')
    title = Label(root, text="Spaced Repetition is Cool!")
    title.grid(row=0, column=0, columnspan=2)
    add_button = Button(root, text='Add', command=add)
    add_button.grid(row=3, column=0)

def create_add_window():
    add_window = Tk()
    add_window.title('Add cards')
    add_window.geometry('400x400')

    # Field Labl
    front_label = Label(add_window, text='Front').grid(row=1, column=0, sticky='W')
    back_label = Label(add_window, text='Back').grid(row=2, column=0, sticky='W')
    # Field Entry
    front_box = Entry(add_window)
    front_box.grid(row=1, column=1)
    back_box = Entry(add_window)
    back_box.grid(row=2, column=1)
    boxes = [front_box, back_box]
    
    clear_button = Button(add_window, text='Clear', command=clear_fields)
    clear_button.grid(row=3, column=1)

def clear_fields():
    front_box.delete(0, END)
    back_box.delete(0, END)
    
create_main_window()

# def add():
#     sql_command = f"""
#         INSERT INTO CARDS (front, back) 
#             VALUES ('{front_box.get()}','{back_box.get()}')
#         """
#     c.execute(sql_command)
#     clear_fields()




root.mainloop()

In [120]:
c.execute('select * from cards')

In [103]:
c.fetchall()

[(1, 'x', 'y', '2020-02-14 02:28:45', '2020-02-14 02:28:45', 0)]

In [ ]:
# Save (commit) the changes
conn.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
conn.close()

# Helper Functions

In [15]:
import pandas as pd

In [6]:
def show_db(table_names=None):
    if table_names is None: table_names = get_names()
    for table_name in table_names:
        print(table_name)
        q = f"select * from {table_name}"
        display(pd.read_sql(q, conn))

In [7]:
def drop_tables(table_names=None):
    if table_names is None: table_names = get_names()
    for table_name in table_names:
        q = f"drop table {table_name}"
        c.execute(q)

In [8]:
def drop_triggers(trigger_names=None):
    if trigger_names is None: trigger_names = get_names('trigger')
    for trigger_name in trigger_names:
        q = f"drop trigger {trigger_name}"
        c.execute(q)

In [9]:
def drop_all():
    drop_triggers()
    drop_tables()

In [10]:
def get_names(type='table'):
    q = f"""
    select name from sqlite_master 
    where type = '{type}' and tbl_name != 'sqlite_sequence'
    """
    c.execute(q)
    names = [o[0] for o in c.fetchall()]
    return names